In [1]:
import tensorflow as tf
import os

# Avoid OOM (Out of Memory) errors by setting GPU Memory Consumption Growth
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)
    
print(f"GPUs Setup: {len(gpus)}")

2025-12-29 20:16:31.615090: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


GPUs Setup: 1


Image processing

In [5]:
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt

data_dir = 'data'
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [4]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))


In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data') # Keras built in dataset creation

Found 305 files belonging to 2 classes.


I0000 00:00:1767020836.375854     911 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
data_iterator = data.as_numpy_iterator() # Cannot iterate through keras created dataset so have to convert it to a numpy iterator

In [ ]:
batch = data_iterator.next()